# 符合Python风格的对象

## 9.1 对象表示形式

repr(): 便于开发者理解的方式返回对象的字符串表示形式;
str(): 以便于用户理解的方式返回对象的字符串表示形式;

`__bytes__` : 调用它获取对象的字节序列表示形式;

`__format__`: 会被内置的format()函数和str.format()方法调用



## 再谈向量类



In [1]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        # 使用float尽早捕获报错，以防传入不正确参数
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

In [2]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [3]:
x,y = v1
x,y

(3.0, 4.0)

In [4]:
v1

Vector2d(3.0, 4.0)

In [5]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [6]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [7]:
abs(v1)

5.0

In [9]:
bool(v1), bool(Vector2d(0,0))

(True, False)

## 备选构造方式

如何将字节序列转换成Vector2d

In [11]:
@classmethod
def frombytes(cls, octets): # 不需要传入self，但是要通过cls传入类本身
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)

## classmethod 与 staticmethod

+ classmethod改变了调用方法的方式，因此类方法的第一个参数是类本身，而不是实例。classmethod最常见的用途是定义备选构造函数。

+ staticmethod装饰器也会改变方法的调用方式，但是第一个参数不是特殊的值。其实，静态方法就是普通的函数，只是碰巧在类的定义体中，而不是在模块层定义。

In [14]:
class Demo:
    @classmethod
    def kclassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args

In [15]:
Demo.kclassmeth()

(__main__.Demo,)

In [17]:
Demo.kclassmeth("spam")

(__main__.Demo, 'spam')

In [16]:
Demo.statmeth()

()

In [18]:
Demo.statmeth("spam")

('spam',)

不管怎么调用，classmethods的方法，返回的第一个都是类。

## 格式化显示

内置的format函数和str.format()方法，是把各个类型的格式化方式委托给相应的`.__format__(format_spec)`方法。

fromat_spec 是格式化说明符，它是：

+ `format(my_obj, format_spec)`的第二个参数，或者
+ `str.format()`方法的格式字符串，{}里代换字段中冒号后面的部分

In [19]:
brl = 1/2.43
format(brl, '0.4f')

'0.4115'

In [20]:
"1 BRL = {rate:0.2f} USD".format(rate=brl)

'1 BRL = 0.41 USD'

In [21]:
format(42, 'b')

'101010'

In [23]:
format(2/3, '.1%')

'66.7%'

如果类没有定义__format__方法，从object继承的方法，会返回str(my_object)。

我们为Vector2d类定义了__str__方法，因此:

In [24]:
v1

Vector2d(3.0, 4.0)

In [25]:
format(v1)

'(3.0, 4.0)'

In [26]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

为了让format接受格式说明符

In [27]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        # 使用float尽早捕获报错，以防传入不正确参数
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components)

In [30]:
format(1.2, '.2e')

'1.20e+00'

## 可散列的Vecto2d



In [31]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        # 使用float尽早捕获报错，以防传入不正确参数
        self.__x = float(x)
        self.__y = float(y)
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

In [32]:
v1 = Vector2d(3,4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(7, 384307168202284039)

要想创建可散列的类型，不一定要实现特性。也不一定要保护视力属性，只需要正确地实现__hash__和__eq__方法即可。
但是，实例的散列值绝不应该变化，因此我们阶级提到了只读特性。

如果定义的类型有标量数值，可能还要实现__int__和__float__方法，以便在某些情况下用于强制转换类型。
此外还有用于支持内置的`complex()`构造函数的`__complex__`方法。



## Python的私有属性和“受保护的”属性

有人编写了一个dog的类，这个类的内部用到了mood实例属性，但是没有将其开放。现在，你创建了Dog类的子类:Beagle

如果你在毫不知情的情况下又创建了名为mood的实例属性，那么在继承的方法中就会把Dog类的mood属性覆盖掉。

为了避免出现这种情况，如果以__mood的形式命名实例属性，Python会把属性名存入实例的__dict__属性中，而且会在前面加上一个下划线和类名。

因此，对Dog类来说，__mood会变成_Dog__mood，对Beagle类来说，会变成_Beagle__mood 这个语言特性叫名称改写

In [33]:
v1 = Vector2d(3,4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [34]:
v1.x

3.0

In [35]:
v1._Vector2d__x

3.0

## 使用__slots__类属性节省空间

Python在各个实例中名为__dict__的字典里存储实例属性，为了使用底层的散列表提升访问速度，字典会消耗大量内存。

如果要处理数百万个属性，通过__slots__类属性，能节省大量内存，方法是让解释器在元组中存储实例属性，而不用字典。

定义__slots__的方式是，创建一个类属性，使用__slots__这个名字，并把它的值设为一个字符串构成的可迭代对象，其中各个元素表示各个实例属性。


In [36]:
from array import array
import math

class Vector2d:
    __slots__ = ('__x', '__y')
    
    typecode = 'd'
    
    def __init__(self, x, y):
        # 使用float尽早捕获报错，以防传入不正确参数
        self.__x = float(x)
        self.__y = float(y)
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

然而，“节省的内存也可能被再次吃掉”。

+  每个子类都要定义__slots__属性，因为解释器会忽略继承的__slots__属性
+ 实例只能拥有__slots__属性, 除非把‘__dict__’加入"__slots__"
+ 如果不把'__weakref_'加入"__slots__"，实例就不能作为弱引用的目标。

In [37]:
v1 = Vector2d(32,4)
v1

Vector2d(32.0, 4.0)

In [39]:
v1.__slots__

('__x', '__y')

## 覆盖类属性

Python有个很独特的特性，类属性可用于为实例属性提供默认值。

Vector2d中有个typecode类属性，所以self.typecode默认获取的是Vector2d.typecode类属性的值。

但是如果为不存在的实例属性赋值，会新建实例属性。假设我们为typecode实例属性赋值，那么同名类属性不受影响。

然而自此之后，实例读取的self.typecode是实例属性typecode，也就是把类属性覆盖了。借助这个特性，我们可以为各个实例的typecode属性定制不同的值。

In [43]:
from array import array
import math

class Vector2d:    
    typecode = 'd'
    
    def __init__(self, x, y):
        # 使用float尽早捕获报错，以防传入不正确参数
        self.__x = float(x)
        self.__y = float(y)
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

In [44]:
v1 = Vector2d(3,4)
v1

Vector2d(3.0, 4.0)

In [45]:
dumped = bytes(v1)
dumped

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [46]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\x00\x00@@\x00\x00\x80@'